In [ ]:
# del session.logger
# del session

import multiprocessing


from train.training_session_v0 import TrainingSessionV0
from config import TrainingConfigV0
from config import SimpleConvConfig

data_partition = {
    "gwilliams2023": {
        "testing_subjects": [19, 20, 21],
        "testing_tasks": [0],
    },
    # "armeini2022": {
    #     "testing_subjects": [],
    #     "testing_tasks": [8, 9],
    # },
}

# model_config = SimpleConvConfig(
#     # Str to list of possible conditions
#     mel_normalization=False,
#     conditions={
#         "study": [],
#         "subject": [],
#     },
#     # Channels
#     in_channels=208,
#     out_channels=128,
#     hidden_dim=384,
#     dropout=0.2,
#     initial_batch_norm=True,
#     # Sensor layout settings
#     layout_dim=2,
#     layout_proj=True,
#     layout_scaling="minmax",
#     # Merger with spatial attn
#     merger=False,
#     merger_emb_type=None,
#     merger_emb_dim=0,
#     merger_channels=0,
#     merger_dropout=False,
#     merger_conditional=None,
#     # Inital
#     initial_linear=384,
#     initial_depth=1,
#     # Conditional layers
#     conditional_layers=False,
#     conditional_layers_dim=None,  # input or hidden_dim
#     # Conv layer overall structure
#     depth=6,
#     kernel_size=3,
#     growth=1.0,
#     dilation_growth=2,
#     dilation_period=5,
#     glu=1,
#     conv_dropout=0.2,
#     dropout_input=0.2,
#     batch_norm=True,
#     half=True,
#     cnn_pos_encoding=False,
#     # Quantizer
#     quantizer=False,
#     num_codebooks=0,
#     codebook_size=0,
#     quantizer_commitment=0,
#     quantizer_temp_init=0,
#     quantizer_temp_min=0,
#     quantizer_temp_decay=0,
#     # Transformers Encoders
#     transformer_input=None,
#     transformer_encoder_emb=None,
#     transformer_encoder_layers=0,
#     transformer_encoder_heads=0,
#     # Transformer Decoders
#     transformer_decoder_emb=None,
#     transformer_decoder_layers=0,
#     transformer_decoder_heads=0,
#     transformer_decoder_dim=0,
# )

model_config = SimpleConvConfig(
    # Str to list of possible conditions
    mel_normalization=False,
    conditions={
        "study": [],
        "subject": [],
    },
    # Channels
    in_channels=208,
    out_channels=128,
    hidden_dim=384,
    dropout=0.2,
    initial_batch_norm=True,
    # Sensor layout settings
    layout_dim=3,
    layout_proj=False,
    layout_scaling="midpoint",
    # Merger with spatial attn
    merger=True,
    merger_emb_type="linear",
    merger_emb_dim=512,
    merger_channels=256,
    merger_dropout=0.2,  # Float
    merger_conditional=None,
    # Inital
    initial_linear=384,
    initial_depth=1,
    # Conditional layers
    conditional_layers=False,
    conditional_layers_dim="input",  # input or hidden_dim
    # Conv layer overall structure
    depth=6,
    kernel_size=3,
    growth=1.0,
    dilation_growth=2,
    dilation_period=5,
    glu=1,
    conv_dropout=0.2,
    dropout_input=0.2,
    batch_norm=True,
    half=True,
    cnn_pos_encoding=False,
    # Quantizer
    quantizer=False,
    num_codebooks=0,
    codebook_size=0,
    quantizer_commitment=0,
    quantizer_temp_init=0,
    quantizer_temp_min=0,
    quantizer_temp_decay=0,
    # Transformers Encoders
    # Transformers Encoders
    transformer_input="continuous",  # concat or quantized or continuous
    transformer_encoder_emb="groupconv",
    transformer_encoder_layers=0,
    transformer_encoder_heads=8,
    # Conformer encoder variant
    rnn_type="conformer",
    depthwise_conv_kernel_size=31,
    use_group_norm=True,
    convolution_first=False,
    # Transformer Decoders
    transformer_decoder_emb=None,
    transformer_decoder_layers=0,
    transformer_decoder_heads=0,
    transformer_decoder_dim=0,
)

config = TrainingConfigV0(
    brain_encoder_config=model_config,
    data_partition=data_partition,
    # Pre-processing parameters
    # Brain
    new_freq=200,
    frequency_bands={"all": (0.5, 80)},
    max_random_shift=1.0,
    window_size=4,
    window_stride=1,
    brain_clipping=None,
    baseline_window=0.5,
    notch_filter=True,
    scaling="both",
    delay=0.15,
    # Hyperparameters
    learning_rate=3e-4,
    weight_decay=1e-4,
    epochs=50,
    batch_size=256,
    use_clip_loss=True,
    use_mse_loss=True,
    alpha=0.6,
    random_test_size=10,
    seed=42,
)

config.learning_rate = 3e-4
config.batch_size = 256

session = TrainingSessionV0(
    config=config,
    studies={study: "audio" for study in data_partition.keys()},
    data_path="data",
    save_path="saves/phase1/objectives/CLIP_MSE_full_model",
    clear_cache=False,
    cache_name="cache/1",
)

try:
    session.train(
        device="cuda",
        buffer_size=30,
        num_workers=(multiprocessing.cpu_count() - 2),
        max_cache_size=400,
        current_epoch=0,
    )
except KeyboardInterrupt as e:
    print("Exited")

Loading Gwilliams2023 with batch type audio
Data partitioned on studies ['gwilliams2023'].
Train: 135, Unseen Task: 12, Unseen Subject: 45, Unseen Both: 4.

SimpleConv initialized with 9485568 parameters, cond: ['study', 'subject']
Merger True, merger channels 256
ConvBlocks: 6, hidden_dim: 384, params 8858112


2025-01-26 20:31:45,477	INFO worker.py:1841 -- Started a local Ray instance.
(raylet) [2025-01-26 20:31:55,470 E 2086704 2086733] file_system_monitor.cc:116: /tmp/ray/session_2025-01-26_20-31-44_405095_2085843 is over 95% full, available space: 14.3925 GB; capacity: 496.03 GB. Object creation will fail if spilling is required.
(raylet) [2025-01-26 20:32:05,519 E 2086704 2086733] file_system_monitor.cc:116: /tmp/ray/session_2025-01-26_20-31-44_405095_2085843 is over 95% full, available space: 14.3923 GB; capacity: 496.03 GB. Object creation will fail if spilling is required.
(raylet) [2025-01-26 20:32:15,527 E 2086704 2086733] file_system_monitor.cc:116: /tmp/ray/session_2025-01-26_20-31-44_405095_2085843 is over 95% full, available space: 14.3923 GB; capacity: 496.03 GB. Object creation will fail if spilling is required.
(raylet) [2025-01-26 20:32:25,536 E 2086704 2086733] file_system_monitor.cc:116: /tmp/ray/session_2025-01-26_20-31-44_405095_2085843 is over 95% full, available space:

Epoch 1 completed. Loss: 2.7535, Clip Loss: 4.0312, MSE Loss: 0.8370
Accuracy: 0.0066, Top 5: 0.0302, Top 10: 0.0581


(raylet) [2025-01-26 20:35:25,825 E 2086704 2086733] file_system_monitor.cc:116: /tmp/ray/session_2025-01-26_20-31-44_405095_2085843 is over 95% full, available space: 14.39 GB; capacity: 496.03 GB. Object creation will fail if spilling is required.
(raylet) [2025-01-26 20:35:35,839 E 2086704 2086733] file_system_monitor.cc:116: /tmp/ray/session_2025-01-26_20-31-44_405095_2085843 is over 95% full, available space: 14.3899 GB; capacity: 496.03 GB. Object creation will fail if spilling is required.


Test unseen_subject completed. Accuracy: 0.0055, Top 5: 0.0280, Top 10: 0.0550


(raylet) [2025-01-26 20:35:45,850 E 2086704 2086733] file_system_monitor.cc:116: /tmp/ray/session_2025-01-26_20-31-44_405095_2085843 is over 95% full, available space: 14.3899 GB; capacity: 496.03 GB. Object creation will fail if spilling is required.


Test unseen_task completed. Accuracy: 0.0043, Top 5: 0.0212, Top 10: 0.0422
Test unseen_both completed. Accuracy: 0.0063, Top 5: 0.0275, Top 10: 0.0650
Testing completed in 0.59m.
Epoch 1 completed in 4.18m. 0.03m per recording.


(raylet) [2025-01-26 20:35:55,868 E 2086704 2086733] file_system_monitor.cc:116: /tmp/ray/session_2025-01-26_20-31-44_405095_2085843 is over 95% full, available space: 14.3896 GB; capacity: 496.03 GB. Object creation will fail if spilling is required.
(raylet) [2025-01-26 20:36:05,887 E 2086704 2086733] file_system_monitor.cc:116: /tmp/ray/session_2025-01-26_20-31-44_405095_2085843 is over 95% full, available space: 14.3888 GB; capacity: 496.03 GB. Object creation will fail if spilling is required.
(raylet) [2025-01-26 20:36:15,907 E 2086704 2086733] file_system_monitor.cc:116: /tmp/ray/session_2025-01-26_20-31-44_405095_2085843 is over 95% full, available space: 14.3888 GB; capacity: 496.03 GB. Object creation will fail if spilling is required.
(raylet) [2025-01-26 20:36:25,914 E 2086704 2086733] file_system_monitor.cc:116: /tmp/ray/session_2025-01-26_20-31-44_405095_2085843 is over 95% full, available space: 14.3886 GB; capacity: 496.03 GB. Object creation will fail if spilling is re

In [ ]:
# 0009-0001-8105-3643@orcid.org
# ecY0uMidk3ppGG7iCgXV3uQPEI5l2nk2qa6n9EA2uRrPK7ZF76rc81nuwpbG1n3MhDrmO8